In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%pip install datasets transformers
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
path = "/content/drive/MyDrive/A3_task1_data_files"

In [3]:
import pandas as pd
from transformers import pipeline
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

In [4]:
path = "/content/drive/MyDrive/A3_task1_data_files"
data = pd.read_csv(path + '/train.csv')
val_data =pd.read_csv(path +'/dev.csv', sep = "\t")
data.dropna()
val_data.dropna()
val_data = val_data.rename(columns = {'setence1':'sentence1'})
val_data.head()
data.head()

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [59]:
data.head()

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [5]:
model_name = "bert-base-uncased"

In [7]:
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
#testing tokenizer
tokenizer(data["sentence1"][0],data["sentence2"][0])

{'input_ids': [101, 1037, 4946, 2003, 2635, 2125, 1012, 102, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
#creating all tokens
tokens = []
for i in range(len(data)):
  tokens.append(tokenizer(data["sentence1"][i],data["sentence2"][i],max_length = 600, truncation = True, padding = "max_length"))
print(tokens[0],tokens[1])

{'input_ids': [101, 1037, 4946, 2003, 2635, 2125, 1012, 102, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
# class CustomDataset(torch.utils.data.Dataset):
#   def __init__(self,dataframe,max_length,model_name):
#     self.data = dataframe
#     self.tokenizer = BertTokenizer.from_pretrained(model_name)
#     self.max_length = max_length

#   def __len__(self):
#     return len(self.data)

#   def __getitem__(self, idx):
#     sentence1 = str(self.data["sentence1"][idx])
#     sentence2 = str(self.data["sentence2"][idx])
#     encoding = self.tokenizer(sentence1,sentence2,max_length = self.max_length, truncation = True, padding = "max_length", return_tensors = "pt", add_special_tokens = True)
#     #print(encoding.keys())
#     label = torch.tensor(self.data["score"][idx])
#     return encoding,label

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, model_name):
        self.data = dataframe
        self.tokenizer = BertTokenizer.from_pretrained(model_name)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence1 = str(self.data.iloc[idx]['sentence1'])
        sentence2 = str(self.data.iloc[idx]['sentence2'])
        label = self.data.iloc[idx]['score']
        inputs = self.tokenizer(sentence1, sentence2, return_tensors="pt", max_length = 512, padding="max_length", truncation=True, add_special_tokens = True)
        return inputs, torch.tensor(label, dtype=torch.long)

    def get_encoded_data(self):
        input_ids = []
        attention_masks = []
        labels = []
        for i in range(self.__len__()):
            input, label = self.__getitem__(i)
            input_ids.append(input['input_ids'])
            attention_masks.append(input['attention_mask'])
            labels.append(label.float()/5.0)

        input_ids = torch.cat(input_ids, dim=0).squeeze(0)
        attention_masks = torch.cat(attention_masks, dim=0).squeeze(0)
        labels = np.array(labels)
        labels = labels.reshape(-1, 1)
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels

    def get_dataloader(self, batch_size):
        input_ids, attention_masks, labels = self.get_encoded_data()
        dataset = TensorDataset(input_ids, attention_masks, labels)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return dataloader

In [7]:
max_length = 600
task1_train_data = CustomDataset(data,model_name)
task1_val_data = CustomDataset(val_data,model_name)
# task1_train_dataloader = DataLoader(task1_train_data,batch_size = 16, shuffle = True)
# task1_val_dataloader = DataLoader(task1_val_data,batch_size = 16, shuffle = True)
task1_train_dataloader = task1_train_data.get_dataloader(16)
task1_val_dataloader = task1_val_data.get_dataloader(16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
bert = BertForSequenceClassification.from_pretrained(model_name)
bert.classifier = nn.Linear(bert.config.hidden_size, 1)
bert = bert.to(device)
optim = torch.optim.Adam(bert.parameters())
loss = nn.MSELoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
bert

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_losses=[]
val_losses=[]
val_predictions = []
for epoch in range(3):
    i = 0
    bert.train()  # Set the model to training mode
    total_train_loss = 0
    all_train_predictions = []
    all_train_targets = []
    for encoding in task1_train_dataloader:
        i += 1
        inputs = encoding[0].to(device)
        #print(device)
        attention_mask = encoding[1].to(device)
        #print(device)
        labels = encoding[2].to(device)
        #print(device)
        optim.zero_grad()
        # inputs = ids.to(device)
        # encoding = encoding.to(device)
        # attention_mask = attention_mask.to(device)
        #label = label.to(device)
        #print(encoding)
        outputs = bert(input_ids = inputs, attention_mask = attention_mask, labels = labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
        total_train_loss+=loss.item()
        #print(outputs)
        #all_train_predictions.extend(outputs["logits"].view(-1).detach().cpu().numpy())
        #all_train_targets.extend(labels.view(-1).cpu().numpy())


    avg_train_loss = total_train_loss / len(task1_train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss}")
    bert.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    all_val_predictions = []
    all_val_targets = []

    with torch.no_grad():
        for val_data in task1_val_dataloader:
            val_inputs,val_attention_mask,val_labels=val_data
            val_inputs = val_inputs.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)
            val_outputs = bert(input_ids=val_inputs, attention_mask=val_attention_mask, labels=val_labels)

            loss=val_outputs.loss
            total_val_loss += loss.item()

            #all_val_predictions.extend(val_outputs["logits"].view(-1).cpu().numpy())
            #all_val_targets.extend(val_labels.view(-1).cpu().numpy())
        avg_val_loss = total_val_loss / len(task1_val_dataloader)
        val_losses.append(avg_val_loss)
    print(f"Epoch {epoch + 1},  Validation Loss: {avg_val_loss}")

Epoch 1, Training Loss: 0.7027289717137313
Epoch 1,  Validation Loss: 0.7048740140769792
Epoch 2, Training Loss: 0.7080192131822517
Epoch 2,  Validation Loss: 0.6736725756655568


In [ ]:
plt.plot(range(3),train_losses,label = "train loss")
plt.plot(range(3),val_losses,label = "val loss")
plt.legend()
plt.show()

In [ ]:
all_val_predictions = []
all_val_targets = []
with torch.no_grad():
  for val_data in task1_val_dataloader:
    val_inputs,val_attention_mask,val_labels=val_data
    val_inputs = val_inputs.to(device)
    val_attention_mask = val_attention_mask.to(device)
    val_labels = val_labels.to(device)
    val_outputs = bert(input_ids=val_inputs, attention_mask=val_attention_mask, labels=val_labels)

    all_val_predictions.extend(val_outputs["logits"].view(-1).cpu().numpy())
    all_val_targets.extend(val_labels.view(-1).cpu().numpy())

In [37]:
print(all_val_predictions)
print(all_val_targets)
print(val_outputs["logits"])

[0.00054858637, 0.0005486087, 0.00054861617, 0.0005485789, 0.00054860127, 0.0005486087, 0.00054857146, 0.0005485938, 0.0005485938, 0.0005486087, 0.0005486087, 0.00054860127, 0.00054860127, 0.00054863107, 0.0005486087, 0.00054860127, 0.00054858637, 0.0005486087, 0.00054861617, 0.0005485938, 0.0005486087, 0.0005486087, 0.00054860127, 0.0005486087, 0.00054860127, 0.0005485789, 0.0005486087, 0.00054860127, 0.0005485789, 0.00054858637, 0.00054861617, 0.00054860127, 0.0005485938, 0.0005486087, 0.00054863107, 0.00054860127, 0.0005485938, 0.0005486087, 0.00054860127, 0.0005486087, 0.0005486087, 0.0005485789, 0.00054858637, 0.0005486087, 0.0005485938, 0.00054860127, 0.0005485938, 0.0005486236, 0.0005486087, 0.0005486236, 0.0005486087, 0.0005485938, 0.00054858637, 0.00054860127, 0.0005485789, 0.00054858637, 0.0005486236, 0.00054860127, 0.00054858637, 0.00054858637, 0.0005486087, 0.0005485938, 0.0005486087, 0.00054858637, 0.00054860127, 0.0005486087, 0.00054860127, 0.0005485938, 0.0005485789, 0.0

In [ ]:
from scipy.stats import pearsonr

pearson_corr, _ = pearsonr(all_val_targets*5, all_val_predictions*5)
print("Pearson Correlation Coefficient on Dev Set:", pearson_corr)

In [ ]:
torch.save(bert.state_dict(),"/content/drive/MyDrive/model1A.pt")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
newbert = BertForSequenceClassification.from_pretrained(model_name)
newbert.classifier = nn.Linear(bert.config.hidden_size, 1)
newbert = newbert.to(device)
newbert.load_state_dict(torch.load("/content/drive/MyDrive/model1A.pt"))